In [ ]:
####核主成分分析

In [ ]:
data = pd.read_csv(r"C:\Users\Administrator\Desktop\table1_end_data.csv")
data = data.drop(columns = "id")
x = data.iloc[:,1:]
y = data.iloc[:,0]
kernels=['linear','poly']
mse_linear = []
mse_poly = []
for i in kernels:
    for j in range(2,30):
        kpca=decomposition.KernelPCA(n_components=j,kernel=i)
        kpca = kpca.fit_transform(x)
        once = cross_val_score(RFR(n_estimators=20,random_state= 0 ),kpca,y,scoring = "neg_mean_squared_error",cv=10).mean()
        if i == "linear" :
            mse_linear.append(once * -1)
        if i == "poly" :
            mse_poly.append(once * -1)
      

color = ["red" , "blue"]       
fig = plt.figure(figsize=[20,5])
mse = [mse_linear , mse_poly ]
mse_char = ["mse_linear" , "mse_poly" ]
for i ,col in enumerate(mse) :
    ax = fig.add_subplot(1,2,i+1)## 两行两列，每个单元显示核函数为 rbf 的 KernelPCA 一组参数的效果图
    ax.plot(range(2,30), col ,color = color[i] ,label = mse_char[i])
    ax.set_xlabel("number of components after dimension reduction")
    ax.set_ylabel(mse_char[i])
    ax.legend(loc="best")
    n = np.argmin(col)+2
    print("{}的mse最小时所需要的降维为{},均方误差最小为{}".format(mse_char[i],n,col[n-2]))
plt.suptitle("linear or poly of KPCA")
plt.show()          
            


    
    
    
fig = plt.figure(figsize=[20,5])
colors = ["red" ,"yellow" ,"green" , "blue"]
Gammas=[0.1,0.5,1,4]
Gammas_char=["gamma = 0.1","gamma = 0.5","gamma = 1","gamma = 4"]# rbf 核的参数组成的列表。每个参数就是 gamma值
for i,gamma in enumerate(Gammas):
    mse_rbf = []
    for j in range(1,10):
        kpca=decomposition.KernelPCA(n_components=j,kernel='rbf',gamma=gamma)
        kpca = kpca.fit_transform(x)
        once = cross_val_score(RFR(n_estimators=20,random_state= 0 ),kpca,y,scoring = "neg_mean_squared_error",cv=10).mean()
        mse_rbf.append(once * -1)
    ax = fig.add_subplot(2,2,i+1)## 两行两列，每个单元显示核函数为 rbf 的 KernelPCA 一组参数的效果图
    ax.plot(range(1,10), mse_rbf ,color = colors[i] ,label = Gammas_char[i])
    ax.set_xlabel("number of components after dimension reduction ")
    ax.set_ylabel("mse_rbf")
    ax.legend(loc="best")
    n = np.argmin(mse_rbf)+1
    print("{}的mse最小时所需要的降维为{},均方误差最小为{}".format(Gammas_char[i],n,mse_rbf[n-1]))
plt.suptitle("KPCA-rbf")
plt.show()
    

    
    
#####已经确定选用核函数为poly时用26个变量进行降维处理并导出#############    
    
kpca=decomposition.KernelPCA(n_components=26,kernel="poly")
kpca = kpca.fit_transform(x)
var_poly = ["var1","var2","var3","var4","var5","var6","var7","var8","var9"
       ,"var10","var11","var12","var13","var14","var15","var16","var7"
       ,"var18","var19","var20","var21","var22","var23","var24","var25"
       ,"var26"]
x_fe_poly = pd.DataFrame(kpca,columns = var_poly )
x_fe_poly.to_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
fit_26 = RFR(n_estimators=20,random_state= 0).fit(x_fe_poly,y)
features = x_fe_poly.columns
importances = fit_26.feature_importances_
indices = np.argsort(importances[0:10])  # top 10 features
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='g', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()  

In [ ]:
#####RNN

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, SimpleRNN
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math
data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)


training_set = data.iloc[0:325 - 50, : ].values  # 前(325-50=275)天的数据作为训练集,表格从0开始计数
test_set = data.iloc[325 - 50: , : ].values  # 后50天的数据作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
training_set_scaled_x = sc_x.fit_transform(training_set[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
training_set_scaled_y = sc_y.fit_transform(training_set[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_x = sc_x.transform(test_set[:,2:])  # 利用训练集的属性对测试集进行归一化
test_set_y = sc_y.transform(test_set[:,1:2])  # 利用训练集的属性对测试集进行归一化

x_train = []
y_train = []

x_test = []
y_test = []

# 测试集：csv表格中前325-50=275个数据
# 利用for循环，遍历整个训练集，提取训练集中连续5天的26个降维新变量作为输入特征x_train，第6天的RON_LOSS作为标签，for循环共构建325-50-5=270组数据。

for i in range(5, len(training_set_scaled_x)):
    x_train.append(training_set_scaled_x[i - 5:i, :])
    y_train.append(training_set_scaled_y[i, :])
# 对训练集进行打乱
np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)
# 将训练集由list格式变为array格式
x_train, y_train = np.array(x_train), np.array(y_train)

# 使x_train符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。
# 此处整个数据集送入，送入样本数为x_train.shape[0]即270组数据；输入连续5个的26个降维新变量，预测出第6天的RON_LOSS，循环核时间展开步数为5; 每个时间步送入的特征是连续5天的26个降维新变量，有26个数据，故每个时间步输入特征个数为26
x_train = np.reshape(x_train, (x_train.shape[0], 5, 26))
# 测试集：csv表格中后50天数据
# 利用for循环，遍历整个测试集，提取测试集中连续5天的26个降维新变量作为输入特征x_train，第6天的数据作为RON_LOSS，for循环共构建50-5=45组数据。
for i in range(5, len(test_set_x)):
    x_test.append(test_set_x[i - 5 : i, :])
    y_test.append(test_set_y[i, :])
# 测试集变array并reshape为符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))

model = tf.keras.Sequential([
    SimpleRNN(80, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(100),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_squared_error')  # 损失函数用均方误差
# 该应用只观测loss数值，不观测准确率，所以删去metrics选项，一会在每个epoch迭代显示时只显示loss值
#C:\Users\Administrator\Desktop\
checkpoint_save_path = r"C:\Users\Administrator\Desktop\RNNcheckpoint\RNN_RON_LOSS.ckpt"

if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')

history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])

model.summary()



file = open('C:/Users/Administrator/Desktop/RNN_RON_LOSS_weights.txt', 'w' )  # 参数提取
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = sc_y.inverse_transform(y_test)
# 画出真实数据和预测数据的对比曲线
plt.plot(real_RON_LOSS, color='red', label=' realed RON LOSS')
plt.plot(predicted_RON_LOSS, color='blue', label='Predicted RON LOSS')
plt.title('RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()


##########evaluate##############
# calculate MSE 均方误差 ---> E[(预测值-真实值)^2] (预测值减真实值求平方后求均值)
mse = mean_squared_error(predicted_RON_LOSS, real_RON_LOSS)
# calculate RMSE 均方根误差--->sqrt[MSE]    (对均方误差开方)
rmse = math.sqrt(mean_squared_error(predicted_RON_LOSS, real_RON_LOSS))
# calculate MAE 平均绝对误差----->E[|预测值-真实值|](预测值减真实值求绝对值后求均值）
mae = mean_absolute_error(predicted_RON_LOSS, real_RON_LOSS)
print('均方误差: %.6f' % mse)
print('均方根误差: %.6f' % rmse)
print('平均绝对误差: %.6f' % mae)

data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)
all_samples = data.iloc[: , : ].values  # 所有样本的特征作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
test_set_scaled_x = sc_x.fit_transform(all_samples[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_scaled_y = sc_y.fit_transform(all_samples[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化

x_test = []
y_test = []

for i in range(5, len(test_set_scaled_x)):
    x_test.append(test_set_scaled_x[i - 5:i, :])
    y_test.append(test_set_scaled_x[i, :])

# 将训练集由list格式变为array格式
x_test, y_test = np.array(x_test), np.array(y_test)

x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))

################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = all_samples[5:,1:2]

# 画出真实数据和预测数据的对比曲线
plt.plot(range(320,0,-1),real_RON_LOSS.ravel(), color='red', label=' realed RON LOSS')
plt.plot(range(320,0,-1),predicted_RON_LOSS.ravel(), color='blue', label='Predicted RON LOSS')
plt.title(' RNN of RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()

In [ ]:
#####LSTM

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, LSTM
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math



data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)


training_set = data.iloc[0:325 - 50, : ].values  # 前(2426-300=2126)天的开盘价作为训练集,表格从0开始计数，2:3 是提取[2:3)列，前闭后开,故提取出C列开盘价
test_set = data.iloc[325 - 50: , : ].values  # 后50天的开盘价作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
training_set_scaled_x = sc_x.fit_transform(training_set[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
training_set_scaled_y = sc_y.fit_transform(training_set[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_x = sc_x.transform(test_set[:,2:])  # 利用训练集的属性对测试集进行归一化
test_set_y = sc_y.transform(test_set[:,1:2])  # 利用训练集的属性对测试集进行归一化

x_train = []
y_train = []

x_test = []
y_test = []


# 测试集：csv表格中前325-50=275个数据
# 利用for循环，遍历整个训练集，提取训练集中连续5天的26个降维新变量作为输入特征x_train，第6天的RON_LOSS作为标签，for循环共构建325-50-5=270组数据。

for i in range(5, len(training_set_scaled_x)):
    x_train.append(training_set_scaled_x[i - 5:i, :])
    y_train.append(training_set_scaled_y[i, :])
# 对训练集进行打乱
np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)
# 将训练集由list格式变为array格式
x_train, y_train = np.array(x_train), np.array(y_train)

# 使x_train符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。
# 此处整个数据集送入，送入样本数为x_train.shape[0]即270组数据；输入连续5个的26个降维新变量，预测出第6天的RON_LOSS，循环核时间展开步数为5; 每个时间步送入的特征是连续5天的26个降维新变量，有26个数据，故每个时间步输入特征个数为26
x_train = np.reshape(x_train, (x_train.shape[0], 5, 26))
# 测试集：csv表格中后50天数据
# 利用for循环，遍历整个测试集，提取测试集中连续5天的26个降维新变量作为输入特征x_train，第6天的数据作为RON_LOSS，for循环共构建50-5=45组数据。
for i in range(5, len(test_set_x)):
    x_test.append(test_set_x[i - 5 : i, :])
    y_test.append(test_set_y[i, :])
# 测试集变array并reshape为符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))


model = tf.keras.Sequential([
    LSTM(80, return_sequences=True),
    Dropout(0.2),
    LSTM(100),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_squared_error')  # 损失函数用均方误差
# 该应用只观测loss数值，不观测准确率，所以删去metrics选项，一会在每个epoch迭代显示时只显示loss值
#C:\Users\Administrator\Desktop\
checkpoint_save_path = r"C:\Users\Administrator\Desktop\LSTMcheckpoint\LSTM_RON_LOSS.ckpt"

if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')

history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])

model.summary()

file = open('C:/Users/Administrator/Desktop/LSTM_RON_LOSS_weights.txt', 'w' )  # 参数提取
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = sc_y.inverse_transform(y_test)
# 画出真实数据和预测数据的对比曲线
plt.plot(real_RON_LOSS, color='red', label=' realed RON LOSS')
plt.plot(predicted_RON_LOSS, color='blue', label='Predicted RON LOSS')
plt.title('RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()


##########evaluate##############
# calculate MSE 均方误差 ---> E[(预测值-真实值)^2] (预测值减真实值求平方后求均值)
mse = mean_squared_error(predicted_RON_LOSS, real_RON_LOSS)
# calculate RMSE 均方根误差--->sqrt[MSE]    (对均方误差开方)
rmse = math.sqrt(mean_squared_error(predicted_RON_LOSS, real_RON_LOSS))
# calculate MAE 平均绝对误差----->E[|预测值-真实值|](预测值减真实值求绝对值后求均值）
mae = mean_absolute_error(predicted_RON_LOSS, real_RON_LOSS)
print('均方误差: %.6f' % mse)
print('均方根误差: %.6f' % rmse)
print('平均绝对误差: %.6f' % mae)


data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)
all_samples = data.iloc[: , : ].values  # 所有样本的特征作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
test_set_scaled_x = sc_x.fit_transform(all_samples[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_scaled_y = sc_y.fit_transform(all_samples[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化

x_test = []
y_test = []

for i in range(5, len(test_set_scaled_x)):
    x_test.append(test_set_scaled_x[i - 5:i, :])
    y_test.append(test_set_scaled_x[i, :])

# 将训练集由list格式变为array格式
x_test, y_test = np.array(x_test), np.array(y_test)

x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))

################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = all_samples[5:,1:2]

# 画出真实数据和预测数据的对比曲线
plt.plot(range(320,0,-1),real_RON_LOSS.ravel(), color='red', label=' realed RON LOSS')
plt.plot(range(320,0,-1),predicted_RON_LOSS.ravel(), color='blue', label='Predicted RON LOSS')
plt.title(' LSTM of RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()

In [ ]:
######GRU

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, GRU
import matplotlib.pyplot as plt
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
import math


data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)


training_set = data.iloc[0:325 - 50, : ].values  # 前(2426-300=2126)天的开盘价作为训练集,表格从0开始计数，2:3 是提取[2:3)列，前闭后开,故提取出C列开盘价
test_set = data.iloc[325 - 50: , : ].values  # 后50天的开盘价作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
training_set_scaled_x = sc_x.fit_transform(training_set[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
training_set_scaled_y = sc_y.fit_transform(training_set[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_x = sc_x.transform(test_set[:,2:])  # 利用训练集的属性对测试集进行归一化
test_set_y = sc_y.transform(test_set[:,1:2])  # 利用训练集的属性对测试集进行归一化

x_train = []
y_train = []

x_test = []
y_test = []

# 测试集：csv表格中前325-50=275个数据
# 利用for循环，遍历整个训练集，提取训练集中连续5天的26个降维新变量作为输入特征x_train，第6天的RON_LOSS作为标签，for循环共构建325-50-5=270组数据。

for i in range(5, len(training_set_scaled_x)):
    x_train.append(training_set_scaled_x[i - 5:i, :])
    y_train.append(training_set_scaled_y[i, :])
# 对训练集进行打乱
np.random.seed(7)
np.random.shuffle(x_train)
np.random.seed(7)
np.random.shuffle(y_train)
tf.random.set_seed(7)
# 将训练集由list格式变为array格式
x_train, y_train = np.array(x_train), np.array(y_train)

# 使x_train符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]。
# 此处整个数据集送入，送入样本数为x_train.shape[0]即270组数据；输入连续5个的26个降维新变量，预测出第6天的RON_LOSS，循环核时间展开步数为5; 每个时间步送入的特征是连续5天的26个降维新变量，有26个数据，故每个时间步输入特征个数为26
x_train = np.reshape(x_train, (x_train.shape[0], 5, 26))
# 测试集：csv表格中后50天数据
# 利用for循环，遍历整个测试集，提取测试集中连续5天的26个降维新变量作为输入特征x_train，第6天的RON_LOSS作为标签，for循环共构建50-5=45组数据。
for i in range(5, len(test_set_x)):
    x_test.append(test_set_x[i - 5 : i, :])
    y_test.append(test_set_y[i, :])
# 测试集变array并reshape为符合RNN输入要求：[送入样本数， 循环核时间展开步数， 每个时间步输入特征个数]
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))

model = tf.keras.Sequential([
    GRU(80, return_sequences=True),
    Dropout(0.2),
    GRU(100),
    Dropout(0.2),
    Dense(1)
])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mean_squared_error')  # 损失函数用均方误差
# 该应用只观测loss数值，不观测准确率，所以删去metrics选项，一会在每个epoch迭代显示时只显示loss值
#C:\Users\Administrator\Desktop\
checkpoint_save_path = r"C:\Users\Administrator\Desktop\GRUcheckpoint\GRU_RON_LOSS.ckpt"

if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                                                 save_weights_only=True,
                                                 save_best_only=True,
                                                 monitor='val_loss')

history = model.fit(x_train, y_train, batch_size=64, epochs=50, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])

model.summary()



file = open('C:/Users/Administrator/Desktop/GRU_RON_LOSS_weights.txt', 'w' )  # 参数提取
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = sc_y.inverse_transform(y_test)
# 画出真实数据和预测数据的对比曲线
plt.plot(real_RON_LOSS, color='red', label=' realed RON LOSS')
plt.plot(predicted_RON_LOSS, color='blue', label='Predicted RON LOSS')
plt.title('RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()


##########evaluate##############
# calculate MSE 均方误差 ---> E[(预测值-真实值)^2] (预测值减真实值求平方后求均值)
mse = mean_squared_error(predicted_RON_LOSS, real_RON_LOSS)
# calculate RMSE 均方根误差--->sqrt[MSE]    (对均方误差开方)
rmse = math.sqrt(mean_squared_error(predicted_RON_LOSS, real_RON_LOSS))
# calculate MAE 平均绝对误差----->E[|预测值-真实值|](预测值减真实值求绝对值后求均值）
mae = mean_absolute_error(predicted_RON_LOSS, real_RON_LOSS)
print('均方误差: %.6f' % mse)
print('均方根误差: %.6f' % rmse)
print('平均绝对误差: %.6f' % mae)




data = pd.read_csv(r"C:\Users\Administrator\Desktop\x_fe_poly.csv")
data = data.sort_values('id', ascending = False)
all_samples = data.iloc[: , : ].values  # 所有样本的特征作为测试集

# 归一化
sc_x = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
sc_y = MinMaxScaler(feature_range=(0, 1))  # 定义归一化：归一化到(0，1)之间
test_set_scaled_x = sc_x.fit_transform(all_samples[:,2:])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化
test_set_scaled_y = sc_y.fit_transform(all_samples[:,1:2])  # 求得训练集的最大值，最小值这些训练集固有的属性，并在训练集上进行归一化

x_test = []
y_test = []

for i in range(5, len(test_set_scaled_x)):
    x_test.append(test_set_scaled_x[i - 5:i, :])
    y_test.append(test_set_scaled_x[i, :])

# 将训练集由list格式变为array格式
x_test, y_test = np.array(x_test), np.array(y_test)

x_test = np.reshape(x_test, (x_test.shape[0], 5, 26))

################## predict ######################
# 测试集输入模型进行预测
predicted_RON_LOSS = model.predict(x_test)
# 对预测数据还原---从（0，1）反归一化到原始范围
predicted_RON_LOSS = sc_y.inverse_transform(predicted_RON_LOSS)
# 对真实数据还原---从（0，1）反归一化到原始范围
real_RON_LOSS = all_samples[5:,1:2]

# 画出真实数据和预测数据的对比曲线
plt.plot(range(320,0,-1),real_RON_LOSS.ravel(), color='red', label=' realed RON LOSS')
plt.plot(range(320,0,-1),predicted_RON_LOSS.ravel(), color='blue', label='Predicted RON LOSS')
plt.title(' GRU of RON LOSS Prediction')
plt.xlabel('Time')
plt.ylabel('RON LOSS')
plt.legend()
plt.show()

In [5]:
#### 导包和文件
import pandas as pd
import numpy as np
w_285 = pd.read_csv(r"C:\Users\Administrator\Desktop\285_02.csv")
y_285 = pd.read_csv(r"C:\Users\Administrator\Desktop\285_range_01.csv" )
w_285.head()
y_285.head()

k = 0
k_i = []
k_j = []
missing_285 = []
for j in range(w_285.shape[1]):
    for i in range(w_285.shape[0]):
        if w_285.iloc[i,j] < y_285.iloc[j,3] or w_285.iloc[i,j] > y_285.iloc[j,4]:
            w_285.iloc[i,j] = np.nan
            k_j.append(w_285.columns[j])
            k_i.append(w_285.index[i])
            k+=1
v_i = list(set(k_i))
v_j = list(set(k_j))
print("不在操作范围元素个数{},行数{},列数{} \n 列名{}".format(k,len(v_i),len(v_j),v_j))
col = v_j
for j in col:
    w = w_285.loc[:,j]
    h = np.isnan(w).sum()
    print("列名{},有超出范围的值有{}个".format(j,h))
    if h > 5:
        missing_285.append(j)
print("不在操作范围内大于5的列名{},有{}个".format(missing_285,len(missing_285)))

w_285_copy = w_285
w_285_copy.shape
for i in missing_285:
    w_285_copy.drop(columns = i , inplace = True)
w_285_copy.shape

不在操作范围元素个数200,行数40,列数5 
 列名['S-ZORB.SIS_LT_1001.PV', 'S-ZORB.FT_5201.PV', 'S-ZORB.FC_1101.TOTAL', 'S-ZORB.AI_2903.PV', 'S-ZORB.FT_1204.TOTAL']
列名S-ZORB.SIS_LT_1001.PV,有超出范围的值有40个
列名S-ZORB.FT_5201.PV,有超出范围的值有40个
列名S-ZORB.FC_1101.TOTAL,有超出范围的值有40个
列名S-ZORB.AI_2903.PV,有超出范围的值有40个
列名S-ZORB.FT_1204.TOTAL,有超出范围的值有40个
不在操作范围内大于5的列名['S-ZORB.SIS_LT_1001.PV', 'S-ZORB.FT_5201.PV', 'S-ZORB.FC_1101.TOTAL', 'S-ZORB.AI_2903.PV', 'S-ZORB.FT_1204.TOTAL'],有5个


(40, 338)

In [6]:
#***********285标准化**********
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(w_285_copy)
scaler.mean_
scaler.var_
w_285_copy_std = scaler.transform(w_285_copy)
w_285_copy_std = pd.DataFrame(w_285_copy_std , columns = w_285_copy.columns)
#*******判断285异常值并导出**********
k = 0
k_i = []
k_j = []

for j in range(w_285_copy_std.shape[1]):
    for i in range(w_285_copy_std.shape[0]):
        if w_285_copy_std.iloc[i,j] < -3 or w_285_copy_std.iloc[i,j] > 3:
            w_285_copy.iloc[i,j] = np.nan
            w_285_copy_std.iloc[i,j] = np.nan
            k_j.append(w_285_copy_std.columns[j])
            k_i.append(w_285_copy_std.index[i])
            k+=1
v_i = list(set(k_i))
v_j = list(set(k_j))
print("异常值元素个数{},行数{},列数{} \n 列名{}".format(k,len(v_i),len(v_j),v_j))
col = v_j
for j in col:
    w = w_285_copy_std.loc[:,j]
    h = np.isnan(w).sum()
    print("列名{},有异常值{}个".format(j,h))

异常值元素个数0,行数0,列数0 
 列名[]


In [ ]:
#*********285取平均值********
mean_285 = w_285_copy.mean(axis = 0)
mean_285
w_285_copy.to_csv(r"C:\Users\Administrator\Desktop\285_isnan.csv")

In [7]:
w_313 = pd.read_csv(r"C:\Users\Administrator\Desktop\313_02.csv")
w_313 = w_313.drop(columns = "id")
y_313 = pd.read_csv(r"C:\Users\Administrator\Desktop\313_range_918_11.csv" )
k = 0
k_i = []
k_j = []
missing_313 = []
for j in range(w_313.shape[1]):
    for i in range(w_313.shape[0]):
        if w_313.iloc[i,j] < y_313.iloc[j,4] or w_313.iloc[i,j] > y_313.iloc[j,5]:
            w_313.iloc[i,j] = np.nan
            k_j.append(w_313.columns[j])
            k_i.append(w_313.index[i])
            k+=1
v_i = list(set(k_i))
v_j = list(set(k_j))
print("不在操作范围元素个数{},行数{},列数{} \n 列名{}".format(k,len(v_i),len(v_j),v_j))
col = v_j

for j in col:
    w = w_313.loc[:,j]
    h = np.isnan(w).sum()
    print("列名{},有超出范围的值有{}个".format(j,h))
    if h > 5:
        missing_313.append(j)
print("不在操作范围内大于5的列名{},有{}个".format(missing_313,len(missing_313)))


不在操作范围元素个数283,行数40,列数27 
 列名['S-ZORB.AT_5201.PV', 'S-ZORB.SIS_LT_1001.PV', 'S-ZORB.PC_2401.DACA', 'S-ZORB.FC_2501.PV', 'S-ZORB.PC_2401B.DACA', 'S-ZORB.PDT_2409.DACA', 'S-ZORB.AT-0006.DACA.PV', 'S-ZORB.AI_2903.PV', 'S-ZORB.PT_2502.DACA', 'S-ZORB.PC_2401B.PIDA.OP', 'S-ZORB.PT_2501.DACA', 'S-ZORB.PDC_2607.PV', 'S-ZORB.FT_9402.PV', 'S-ZORB.BS_LT_2401.PV', 'S-ZORB.PC_3101.DACA', 'S-ZORB.PC_2401.PIDA.SP', 'S-ZORB.AT-0012.DACA.PV', 'S-ZORB.TE_1603.DACA', 'S-ZORB.PDI_2301.DACA', 'S-ZORB.PDC_2502.PV', 'S-ZORB.FT_1204.TOTAL', 'S-ZORB.PDI_2801.DACA', 'S-ZORB.PT_9403.PV', 'S-ZORB.PT_6002.PV', 'S-ZORB.PC_6001.PV', 'S-ZORB.PC_2401B.PIDA.SP', 'S-ZORB.PC_2401.PIDA.OP']
列名S-ZORB.AT_5201.PV,有超出范围的值有39个
列名S-ZORB.SIS_LT_1001.PV,有超出范围的值有40个
列名S-ZORB.PC_2401.DACA,有超出范围的值有4个
列名S-ZORB.FC_2501.PV,有超出范围的值有4个
列名S-ZORB.PC_2401B.DACA,有超出范围的值有3个
列名S-ZORB.PDT_2409.DACA,有超出范围的值有2个
列名S-ZORB.AT-0006.DACA.PV,有超出范围的值有7个
列名S-ZORB.AI_2903.PV,有超出范围的值有40个
列名S-ZORB.PT_2502.DACA,有超出范围的值有10个
列名S-ZORB.PC_2401B.PIDA.OP,有超出范围的值有4个

In [9]:
w_313_copy = w_313
w_313_copy.shape
for i in missing_313:
    w_313_copy.drop(columns = i , inplace = True)
w_313_copy.shape
#***********313标准化并导出**********
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(w_313_copy)
scaler.mean_
scaler.var_
w_313_copy_std = scaler.transform(w_313_copy)
w_313_copy_std = pd.DataFrame(w_313_copy_std , columns = w_313_copy.columns)
#*******判断313异常值********
k = 0
k_i = []
k_j = []
for j in range(w_313_copy_std.shape[1]):
    for i in range(w_313_copy_std.shape[0]):
        if w_313_copy_std.iloc[i,j] < -3 or w_313_copy_std.iloc[i,j] > 3:
            w_313_copy.iloc[i,j] = np.nan
            w_313_copy_std.iloc[i,j] = np.nan
            k_j.append(w_313_copy_std.columns[j])
            k_i.append(w_313_copy_std.index[i])
            k+=1
v_i = list(set(k_i))
v_j = list(set(k_j))
print("异常值元素个数{},行数{},列数{} \n 列名{}".format(k,len(v_i),len(v_j),v_j))

异常值元素个数63,行数26,列数50 
 列名['S-ZORB.FC_1203.PV', 'S-ZORB.PT_2101.PV', 'S-ZORB.FT_1204.DACA.PV', 'S-ZORB.CAL.LINE.PV', 'S-ZORB.PDT_2409.DACA', 'S-ZORB.FC_3103.PV', 'S-ZORB.PT_1201.PV', 'S-ZORB.TE_1107.DACA', 'S-ZORB.LT_1501.DACA', 'S-ZORB.FT_1204.PV', 'S-ZORB.PT_2106.DACA.PV', 'S-ZORB.PT_9402.PV', 'S-ZORB.PC_2105.PV', 'S-ZORB.AT-0010.DACA.PV', 'S-ZORB.PDT_1003.DACA', 'S-ZORB.PC_1202.PV', 'S-ZORB.PC_2401B.PIDA.OP', 'S-ZORB.PDT_2503.DACA', 'S-ZORB.FT_9401.PV', 'S-ZORB.AC_6001.PV', 'S-ZORB.PDC_2607.PV', 'S-ZORB.FC_2801.PV', 'S-ZORB.TE_1101.DACA', 'S-ZORB.PT_7107B.DACA', 'S-ZORB.ZT_2634.DACA', 'S-ZORB.PT_7103B.DACA', 'S-ZORB.PDT_2606.DACA', 'S-ZORB.TE_5008.DACA', 'S-ZORB.TE_1107.DACA.PV', 'S-ZORB.PT_7505B.DACA', 'S-ZORB.PT_1601.DACA', 'S-ZORB.FC_3101.PV', 'S-ZORB.PC_1301.PV', 'S-ZORB.LT_1002.DACA', 'S-ZORB.TE_1101.DACA.PV', 'S-ZORB.FT_9102.PV', 'S-ZORB.FT_3702.DACA', 'S-ZORB.PT_6002.PV', 'S-ZORB.PDT_3502.DACA', 'S-ZORB.PT_2607.DACA', 'S-ZORB.FC_1202.PV', 'S-ZORB.TE_1106.DACA', 'S-ZORB.FT_9302.